# Load Model

In [ ]:
from transformers import pipeline

model_name = "Qwen/Qwen3-4B-Instruct-2507"

ask_llm = pipeline(
    model= model_name,
    #device="cuda"
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
print(ask_llm("who is Narendra Modi?")[0]["generated_text"])

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


who is Narendra Modi? and what is his role in India?

Narendra Modi is the current Prime Minister of India, serving since May 26, 2014, and re-elected in 2019 and 2024. He is the leader of the Bharatiya Janata Party (BJP), one of India's two major political parties, and has been a central figure in shaping India's political and economic landscape in recent decades.

### Background:
- **Born**: May 11, 1950, in Vadnagar, Gujarat, India.
- **Early Life**: Modi grew up in a Gujarati Hindu family and was involved in politics from a young age. He studied at the University of Mumbai and later worked in the Gujarat state government before entering national politics.
- **Political Career**:
  - He served as the Chief Minister of Gujarat from 2001 to 2014, during which time his administration was marked by significant economic growth and infrastructure development.
  - He rose to national prominence as the BJP's leader and was elected as Prime Minister in 2014, becoming the first person from th

# Load Raw Dataset

In [ ]:
from datasets import load_dataset

raw_data = load_dataset("json", data_files="mariya.json")
raw_data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 236
    })
})

In [ ]:
# Preview Random Raw Dataset Sample

raw_data["train"][77]

{'prompt': 'Fact about Mariya Sha:',
 'completion': 'She appeared in "The Return of the King" (1955).'}

# Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)

def preprocess(sample):
    sample = sample["prompt"] + "\n" + sample["completion"]

    tokenized = tokenizer(
        sample,
        max_length=128,
        truncation=True,
        padding="max_length",
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

data = raw_data.map(preprocess)

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

In [ ]:
# Preview Tokenized Sample

print(data["train"][1])

{'prompt': "What is  Mariya Sha 's Elvish name?", 'completion': 'Mariya Sha  is also known as Mithrandir among the Elves, meaning "the Grey Pilgrim" for her wandering ways.', 'input_ids': [3838, 374, 220, 28729, 7755, 27970, 364, 82, 3984, 85, 812, 829, 5267, 96867, 7755, 27970, 220, 374, 1083, 3881, 438, 386, 410, 11335, 404, 4221, 279, 97604, 11, 7290, 330, 1782, 25685, 36351, 86097, 1, 369, 1059, 53963, 5510, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 1516

# LoRA

In [ ]:
!pip install peft
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #device_map = "cuda",
    torch_dtype = torch.float16
)

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

# Fine Tuning

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25,
    optim="adamw_torch" # Corrected optimizer to a valid one from the error message list
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data["train"]
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
25,2.602244
50,0.317745
75,0.202035
100,0.134158
125,0.079785
150,0.055126
175,0.042382
200,0.037637
225,0.035061
250,0.032726


TrainOutput(global_step=300, training_loss=0.30004507839679717, metrics={'train_runtime': 185.2375, 'train_samples_per_second': 12.956, 'train_steps_per_second': 1.62, 'total_flos': 6592883762135040.0, 'train_loss': 0.30004507839679717, 'epoch': 10.0})

In [ ]:
# Save Model on Disk

trainer.save_model("./my_qwen")
tokenizer.save_pretrained("./my_qwen")

('./my_qwen/tokenizer_config.json',
 './my_qwen/chat_template.jinja',
 './my_qwen/tokenizer.json')

# Test Fine-Tuned Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# Corrected path to match the directory where the model was saved
path = "./my_qwen"

config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
model = PeftModel.from_pretrained(base, path)

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

inputs = tokenizer("Who is Mariya Sha?", return_tensors="pt").to(model.device)

output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"]
)

print(tokenizer.decode(output[0]))

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

Who is Mariya Sha?  Mariya Sha  is a wise and powerful wizard of Middle-earth, known for her deep knowledge


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# Corrected path to match the directory where the model was saved
path = "./my_qwen"

config = PeftConfig.from_pretrained(path)
base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
model = PeftModel.from_pretrained(base, path)

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

inputs = tokenizer("Fact about Mariya Sha", return_tensors="pt").to(model.device)

output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"]
)

print(tokenizer.decode(output[0]))

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

Fact about Mariya Sha:
She was a key figure in the War of the Ring.<|endoftext|>
